# Building a Command-Line Solitaire Game in Python

Welcome! In this notebook, you'll learn how to build a complete Solitaire game that runs in your terminal. We'll break the project into small, easy-to-understand steps. You'll learn about:

- **Classes and objects** (how to model cards and decks)
- **Game logic** (how Solitaire works)
- **Displaying the game** (making it look nice in the terminal)
- **User input and the main game loop**

## 1. The Card Class

A card in Solitaire has a **suit** (Hearts, Diamonds, Spades, Clubs), a **value** (Ace through King), and can be **face up** or **face down**.

We'll use a Python class to represent a card. This lets us program all the information and behaviors (like flipping the card) in a single object.

In [4]:
# Copy this from your card.py
class Card:
    def __init__(self, suit, value):
        # The suit of the card (e.g., 'Hearts', 'Spades')
        self.suit = suit
        # The value of the card (1 = Ace, 11 = Jack, 12 = Queen, 13 = King)
        self.value = value
        # Whether the card is face up (visible) or face down
        self.face_up = False

    def __str__(self):
        # If the card is face down, show a hidden card symbol
        if not self.face_up:
            return "[X]"
        # Map special values to their display characters (A, J, Q, K)
        value_map = {1: 'A', 11: 'J', 12: 'Q', 13: 'K'}
        display_value = value_map.get(self.value, str(self.value))
        suit_symbols = {
            'Hearts': '♥',
            'Diamonds': '♦',
            'Spades': '♠',
            'Clubs': '♣'
        }
        return f"[{display_value}{suit_symbols[self.suit]}]"

    def color(self):
        # Returns 'red' for Hearts/Diamonds, 'black' for Spades/Clubs
        if self.suit in ['Hearts', 'Diamonds']:
            return 'red'
        else:
            return 'black'

    def flip(self):
        # Flips the card (face up <-> face down)
        self.face_up = not self.face_up

**Explanation:**  
- The `__init__` method sets up the card's suit, value, and whether it's face up.
- The `__str__` method controls how the card is printed.
- The `color` method tells us if the card is red or black.
- The `flip` method turns the card over.

**Try it yourself:**  
Create a card and print it. Try flipping it and printing again!

In [ ]:
# Example:
card1 = Card('Hearts', 1)  # Ace of Hearts
print(card1)  # Should show [A♥]

card1.flip()  # Flip the Ace of Hearts to face up
print(card1)  # Should now show [A♥]

## 2. The Deck Class

A deck is just a collection of 52 cards (one for each suit and value). We'll use a class to manage the deck: building it, shuffling it, and dealing cards from it.

In [8]:
# Copy this from your deck.py
import random

class Deck:
    def __init__(self):
        self.cards = []
        self.build()

    def build(self):
        # Create all 52 cards and add them to the deck
        suits = ['Hearts', 'Diamonds', 'Spades', 'Clubs']
        values = range(1, 14)  # 1 (Ace) to 13 (King)
        for suit in suits:
            for value in values:
                self.cards.append(Card(suit, value))

    def shuffle(self):
        # Shuffle the deck randomly
        random.shuffle(self.cards)

    def deal(self):
        # Remove and return the top card, or None if the deck is empty
        if len(self.cards) > 0:
            return self.cards.pop()
        return None

**Explanation:**  
- The `build` method creates all 52 cards.
- The `shuffle` method mixes up the cards.
- The `deal` method gives you the top card from the deck.

**Try it yourself:**  
Create a deck, shuffle it, and deal a few cards. Print them to see what you get!

In [ ]:
# Example usage:
deck = Deck()
deck.shuffle()  # Shuffle the deck
card = deck.deal()  # Deal a card from the deck
print(card)  # Should show the dealt card, e.g., [10♠] or [A♦]

# Now try to shuffle the deck again and deal another card!

## 3. The SolitaireGame Class

This class manages the entire game: the tableau (columns), foundations, reserve (stock), and waste piles. It also contains all the rules for moving cards.

Let's start with the setup and then add methods for moves.

In [12]:
# The complete SolitaireGame class with all main logic and comments.
from deck import Deck

class SolitaireGame:
    """
    The SolitaireGame class contains all the logic and state for a game of Solitaire.
    It manages the deck, tableau columns, foundations, reserve (stock), and waste piles.
    """

    def __init__(self):
        # Create and shuffle a new deck
        self.deck = Deck()
        self.deck.shuffle()
        # Set up 7 tableau columns (where cards are played)
        self.columns = [[] for _ in range(7)]
        # Set up 4 foundations (one for each suit)
        self.foundations = {suit: [] for suit in ['Hearts', 'Diamonds', 'Spades', 'Clubs']}
        # The reserve (stock) pile and waste (face-up discard)
        self.reserve = []
        self.waste = []
        # Counter for how many times the reserve has been recycled
        self.reserve_recycles = 0
        # Deal cards to columns and reserve
        self.setup_columns()
        self.setup_reserve()

    def setup_columns(self):
        """
        Deal cards to columns according to solitaire rules:
        Column 1 gets 1 card, column 2 gets 2, ..., column 7 gets 7 cards.
        Only the top card in each column is face up.
        """
        # Loop through each column index from 0 to 6 (7 columns)
        # Each column i gets i + 1 cards, with the last card face up
        for i in range(7):
            for j in range(i + 1):
                # Deal a card from the deck
                card = self.deck.deal()
                # If the card is not None, add it to the column
                if card is not None:
                    # If this is the last card in the column, flip it face up
                    if j == i:  # Last card in column is face up
                        card.flip()
                    self.columns[i].append(card)

    def setup_reserve(self):
        """
        All remaining cards after dealing columns go to the reserve (stock) pile.
        """
        while len(self.deck.cards) > 0:
            card = self.deck.deal()
            self.reserve.append(card)

    def draw_from_reserve(self):
        """
        Draw the top card from the reserve (stock) pile and place it face up on the waste pile.
        If the reserve is empty, recycle the waste pile back into the reserve.
        """
        # Check if there are cards in the reserve
        if self.reserve:
            # Draw the top card from the reserve and place it face up in the waste pile
            card = self.reserve.pop()
            # Ensure the card is face up
            card.face_up = True
            # Add the card to the waste pile
            self.waste.append(card)
        else:
            # Recycle the waste pile if reserve is empty
            if self.waste:
                # Move all cards from the waste pile back to the reserve
                while self.waste:
                    # Pop the top card from the waste pile
                    card = self.waste.pop()
                    # Ensure the card is face down when recycled
                    card.face_up = False
                    # Add the card back to the reserve
                    self.reserve.append(card)
                # Increment the recycle counter for statistics in end game
                self.reserve_recycles += 1

    def move_waste_to_column(self, col_num):
        """
        Move the top card from the waste pile to a tableau column, if the move is valid.
        Returns True if the move was successful, False otherwise.
        """
        # Check if waste pile is empty, if so, return False
        if not self.waste:
            return False
        # Get the top card from the waste pile
        card = self.waste[-1]
        # Check if the column exists and can accept the card
        target_col = self.columns[col_num]
        # Check if the move to the column is valid
        if self.can_move_to_column(card, target_col):
            # If the move is valid, pop the card from the waste pile and add it to the column
            self.waste.pop()
            # If the column is empty, the card is added face up
            target_col.append(card)
            return True
        # If the move is not valid, return False
        return False

    def move_waste_to_foundation(self):
        """
        Move the top card from the waste pile to the appropriate foundation, if the move is valid.
        Returns True if the move was successful, False otherwise.
        """
        # Check if there is a card in the waste pile, if not, return False
        if not self.waste:
            return False
        # Get the top card from the waste pile
        card = self.waste[-1]
        # Get the foundation for the card's suit
        foundation = self.foundations[card.suit]
        # Check if the move to foundation is valid
        if self.can_move_to_foundation(card, foundation):
            # pop the card from the waste pile and add it to the foundation
            self.waste.pop()
            foundation.append(card)
            return True
        # If the move is not valid, return False
        return False

    def move_to_foundation(self, col_num):
        """
        Move the top card from a tableau column to the appropriate foundation, if the move is valid.
        Returns True if the move was successful, False otherwise.
        """
        # Check if the column exists and has a face-up card
        column = self.columns[col_num]
        if not column or not column[-1].face_up:
            return False
        
        # Get the top card of the column (selecting the last card with -1 index)
        card = column[-1]
        # Get the foundation for the card's suit
        foundation = self.foundations[card.suit]
        # Check if the move to foundation is valid
        if self.can_move_to_foundation(card, foundation):
            column.pop()
            foundation.append(card)
            # Flip the next card in the column if needed
            if column and not column[-1].face_up:
                column[-1].flip()
            return True
        return False

    def move_between_columns(self, source_col, source_index, target_col):
        """
        Move a sequence of face-up cards from one tableau column to another, if the move is valid.
        source_col: index of the source column
        source_index: index of the first card to move in the source column
        target_col: index of the target column
        Returns True if the move was successful, False otherwise.
        """
        # Get the source and target columns
        source = self.columns[source_col]
        target = self.columns[target_col]
        # Get all cards from the source column starting from source_index
        moving_cards = source[source_index:]
        # Check if there are any cards to move and if the first card is face up
        if not moving_cards or not moving_cards[0].face_up:
            return False
        if self.can_move_to_column(moving_cards[0], target):
            self.columns[target_col].extend(moving_cards)
            del self.columns[source_col][source_index:]
            # Flip the next card in the source column if needed
            if self.columns[source_col] and not self.columns[source_col][-1].face_up:
                self.columns[source_col][-1].flip()
            return True
        return False

    def can_move_to_column(self, card, column):
        """
        Check if a card can be moved to the given tableau column according to Solitaire rules.
        - If the column is empty, only a King can be placed.
        - Otherwise, the card must be one less in value and of opposite color.
        """
        # Check if the column is empty
        if not column:
            return card.value == 13  # King
        # Get the top card of the column
        top_card = column[-1]
        # Return True if the card can be placed on the top card of the column
        return (top_card.face_up and
                card.color() != top_card.color() and
                card.value == top_card.value - 1)

    def can_move_to_foundation(self, card, foundation):
        """
        Check if a card can be moved to the given foundation pile according to Solitaire rules.
        - If the foundation is empty, only an Ace can be placed.
        - Otherwise, the card must be the same suit and one higher in value.
        """
        # Check if the foundation is empty
        if not foundation:
            # Only an Ace can be placed on an empty foundation
            return card.value == 1  # Ace
        # Get the top card of the foundation
        top_card = foundation[-1]
        # Return True if the card can be placed on the top card of the foundation
        return card.suit == top_card.suit and card.value == top_card.value + 1

    def check_win(self):
        """
        Check if the player has won the game (all foundations have 13 cards).
        """
        # Check if all foundations have exactly 13 cards
        for foundation in self.foundations.values():
            # If any foundation does not have 13 cards, the game is not won
            if len(foundation) != 13:
                return False
        # If all foundations have 13 cards, the game is won
        return True

**Explanation:**  
- The `columns` are where you play cards.
- The `foundations` are where you build up each suit from Ace to King.
- The `reserve` is the face-down stock pile.
- The `waste` is the face-up pile where drawn cards go.

**Try it yourself:**  
Create a `SolitaireGame` and inspect its columns, reserve, and waste.

In [ ]:
# Example: Inspecting columns, reserve, and waste in SolitaireGame

# Create a new SolitaireGame instance
game = SolitaireGame()

# Inspect the tableau columns
print("Tableau columns (first card in each column):")
for i, col in enumerate(game.columns):
    if col:
        print(f"Column {i+1}: {col[-1]} (top card)")
    else:
        print(f"Column {i+1}: empty")

# Inspect the reserve (stock) pile
print(f"\nReserve pile has {len(game.reserve)} cards.")

# Inspect the waste pile
if game.waste:
    print(f"Waste pile top card: {game.waste[-1]}")
else:
    print("Waste pile is empty.")

## 4. The Display Functions

We want to show the game state in the terminal. We'll write functions to print the columns, foundations, reserve, and waste in a readable way.

In [23]:
import re # Import the regular expression module

# Regex (short for regular expression) is a sequence of characters that defines a search 
# pattern. It is used for matching, searching, and manipulating text based on specific patterns

# Escape codes (or escape sequences) are special combinations of characters used 
# to control formatting, color, cursor position, or other output options on terminals 
# and text displays. They usually start with an "escape" character (ASCII 27, written 
# as \x1b or \033 in Python) followed by other characters that specify the command.

# ANSI_ESCAPE written below is a regular expression used to find ANSI escape codes in strings.
# These codes are used to add color or formatting in terminal output, but they are not printed in screen,
# so they do not take up space when displayed. However, they still count as characters in string length calculations.
# When aligning text (like columns of cards), these invisible codes can cause misalignment if counted as characters.
# By removing ANSI codes before measuring string length, we ensure proper alignment regardless of color formatting.
ANSI_ESCAPE = re.compile(r'\x1b\[[0-9;]*m')

def pad_ansi(s, width):
    """
    Pad string s to a given width, ignoring ANSI escape codes.
    This ensures columns align even when using colored output.
    """
    # Remove ANSI escape codes from the string to calculate real length
    real_len = len(ANSI_ESCAPE.sub('', s))
    return s + ' ' * (width - real_len)

# Suit symbols with ANSI color codes for display
suit_symbols = {
    'Hearts': '\033[91m♥\033[0m',    # Red heart
    'Diamonds': '\033[91m♦\033[0m',  # Red diamond
    'Spades': '\033[37m♠\033[0m',    # Black spade
    'Clubs': '\033[37m♣\033[0m'      # Black club
}

def clear_screen():
    """
    Clear the terminal screen using an ANSI escape code.
    """
    # \033[2J clears the screen, \033[H moves the cursor to the home position
    print("\033[2J\033[H", end="")
    
def colored_card(card):
    """
    Return a string representation of a card, with color for suits and face-down cards.
    """
    if not card.face_up:
        return "\033[30m[X]\033[0m"  # Gray for face-down
    value_map = {1: 'A', 11: 'J', 12: 'Q', 13: 'K'}
    display_value = value_map.get(card.value, str(card.value))
    card_str = f"{display_value}{suit_symbols[card.suit]}"
    if card.suit in ['Hearts', 'Diamonds']:
        return f"[\033[91m{card_str}\033[0m]"  # Red for hearts/diamonds
    else:
        return f"[\033[37m{card_str}\033[0m]"  # White for spades/clubs

def display_game(game):
    """
    Display the current state of the game, including foundations, waste, reserve, and tableau columns.
    """
    clear_screen()
    print("=== SOLITAIRE ===")

    # Display foundations (top card of each suit's pile)
    print("\nFoundations:")
    for suit, foundation in game.foundations.items():
        # Display the top card of the foundation, or empty if no cards
        if foundation:
            print(f"{suit_symbols[suit]}: {colored_card(foundation[-1])}", end=" ")
        else:
            print(f"{suit_symbols[suit]}: [  ]", end=" ")
    print("\n")

    # Display the waste pile (top card of waste)
    print("Waste:", end=" ")
    # If there is a card in the waste, display it; otherwise, show empty
    if game.waste:
        # Display the top card of the waste pile
        print(colored_card(game.waste[-1]), end=" ")
    else:
        # If the waste pile is empty, show empty
        print("[  ]", end=" ")
    print(f"(Remaining: {len(game.reserve)} cards in reserve)\n")

    # Display compact column headers (C1, C2, ...)
    col_width = 6
    print("".join(pad_ansi(f"C{i+1}", col_width) for i in range(7)))

    # Display tableau columns with fixed width, left-aligned
    max_height = max(len(col) for col in game.columns)
    # Print each row of the tableau
    for row in range(max_height):
        # Print each column's card in the current row
        for col in range(7):
            # If the row index is within the column's height, display the card
            if row < len(game.columns[col]):
                card_display = colored_card(game.columns[col][row])
                print(pad_ansi(card_display, col_width), end="")
            else:
                print(" " * col_width, end="")
        print()  # New line after each row

    # Print controls for the user
    print("\nControls:")
    print("d - draw from reserve")
    print("m<source><target> - move between columns (e.g., m13 moves from C1 to C3)")
    print("mw<column> - move from waste to column (e.g., mw3 moves waste to C3)")
    print("f<column> - move to foundation (e.g., f1 moves from C1 to foundation)")
    print("fw - move from waste to foundation")
    print("q - quit")

**Try it yourself:**  
Create a game and call `display_game(game)` to see the initial setup!

In [ ]:
# Example: Using display_game() to show the current game state

# Create a new SolitaireGame instance
game = SolitaireGame()

# Display the initial state of the game
display_game(game)

## 5. The Main Game Loop

The main loop ties everything together: it displays the game, takes user input, and calls the appropriate methods to update the game state.

In [ ]:
from game import SolitaireGame  # Import the SolitaireGame class from the game module
from display import display_game, print_win_screen  # Import the display_game and print_win_screen functions to show the game state

def main():
    while True:
        game = SolitaireGame()
        while True:
            display_game(game)  # Show the current state of the game

            # Check if the player has won the game
            if game.check_win():
                print_win_screen(game)
                user = input()
                if user.lower().strip() == 'q':
                    print("Goodbye!")
                    return
                else:
                    break  # restart

            # Get a command from the user (convert to lowercase and remove spaces)
            command = input("Enter command: ").lower().strip()

            # If the user wants to quit, exit the loop
            if command == 'q':
                print("Goodbye!")
                return

            # If the user wants to restart, break inner loop
            if command == 'r':
                break

            # If the user wants to draw a card from the reserve (stock) pile
            elif command == 'd':
                game.draw_from_reserve()

            # If the user wants to move the waste card to a foundation pile
            elif command.startswith('fw'):
                game.move_waste_to_foundation()

            # If the user wants to move a card from a column to a foundation pile
            elif command.startswith('f'):
                try:
                    # Get the column number from the command (e.g., 'f1' means column 1)
                    col_num = int(command[1:]) - 1  # Convert to 0-based index
                    if 0 <= col_num < 7:
                        game.move_to_foundation(col_num)
                except (ValueError, IndexError):
                    pass  # Ignore invalid input

            # If the user wants to move the waste card to a column
            elif command.startswith('mw'):
                col_str = command[2:]  # Get the column number after 'mw'
                try:
                    col_num = int(col_str) - 1  # Convert to 0-based index
                    if 0 <= col_num < 7:
                        game.move_waste_to_column(col_num)
                except (ValueError, IndexError):
                    pass  # Ignore invalid input

            # If the user wants to move cards between columns (e.g., 'm13' moves from column 1 to 3)
            elif command.startswith('m'):
                try:
                    if len(command) == 3:
                        source_col = int(command[1]) - 1  # Source column (0-based)
                        target_col = int(command[2]) - 1  # Target column (0-based)
                        if 0 <= source_col < 7 and 0 <= target_col < 7:
                            source = game.columns[source_col]
                            # Find the first face-up card in the source column
                            source_index = next((i for i, card in enumerate(source) if card.face_up), None)
                            if source_index is not None:
                                game.move_between_columns(source_col, source_index, target_col)
                except ValueError:
                    pass  # Ignore invalid input

# This code runs the main() function if the script is executed directly (terminal command python)
if __name__ == "__main__":
    main()

**Explanation:**  
- The loop displays the game, checks for a win, and processes user commands.
- Commands let you draw from the reserve, move cards, or quit the game.

## Instructions

In [3]:
# Python code cell
from IPython.display import Markdown, display

with open("README.md", "r") as f:
    content = f.read()
display(Markdown(content))

# Solitaire (Console Version)

A classic Solitaire (Klondike) card game playable entirely in your terminal, written in Python.

---

## How to Start the Project

1. **Clone or Download the Repository**
   - Download the project folder and enter the main folder:
        ```
     cd Solitaire
     ```

2. **Install Python**
   - Make sure you have Python 3.7 or newer installed.
   - Check with:
     ```
     python3 --version
     ```

3. **(Optional) Create a Virtual Environment**
   - Recommended for dependency management:
     ```
     python3 -m venv venv
     source venv/bin/activate
     ```

4. **Install Dependencies**
   - Install dependencies with:
     ```
     pip install -r requirements.txt
     ```
   - *Note: This project uses only the Python standard library, so this step may be skipped.*

5. **Run the Game**
     ```
     python3 main.py
     ```
---

## Game User Instructions

### **Goal**
Move all cards to the four foundation piles (one for each suit), building up from Ace to King.

### **Game Layout**
- **Tableau:** 7 columns of cards, only the last card in each column is face up.
- **Reserve (Stock):** Remaining cards, face down.
- **Waste:** Cards drawn from the reserve, face up.
- **Foundations:** Four piles, one for each suit, to build from Ace to King.

### **Controls**
Type commands and press Enter:

| Command         | Action                                                        |
|-----------------|--------------------------------------------------------------|
| `d`             | Draw a card from the reserve (stock) to the waste pile       |
| `mXY`           | Move cards from column X to column Y (e.g., `m13` moves from C1 to C3) |
| `mwY`           | Move the top waste card to column Y (e.g., `mw3` to C3)      |
| `fX`            | Move the top card from column X to its foundation (e.g., `f1`)|
| `fw`            | Move the top waste card to its foundation                    |
| `r`             | Restart the game                                             |
| `q`             | Quit the game                                                |

- **Columns are numbered 1–7.**
- **You can only move cards according to Solitaire rules:**
  - Cards in tableau must alternate colors and descend in value.
  - Only Kings (or sequences starting with King) can be placed in empty columns.
  - Foundations must be built up by suit from Ace to King.
  - Only one card is drawn from the reserve at a time; when empty, the waste is recycled.

---

## Code Structure and Class Descriptions

### **Modules / Files**
- `card.py` – Defines the `Card` class (suit, value, face up/down, color, flip).
- `deck.py` – Defines the `Deck` class (builds, shuffles, and deals cards).
- `game.py` – Contains the `SolitaireGame` class (game state and logic).
- `display.py` – Functions for displaying the game state and win screen.
- `main.py` – The main game loop and user input handling.
- `Solitaire_game.ipynb` – Jupyter notebook version with explanations and code.

### **Key Classes and Functions**

#### **Card**
- Represents a single playing card.
- Attributes: `suit`, `value`, `face_up`.
- Methods: `flip()`, `color()`, `__str__()`.

#### **Deck**
- Manages a collection of 52 cards.
- Methods: `build()`, `shuffle()`, `deal()`.

#### **SolitaireGame**
- Manages the entire game state:
  - `columns`: 7 tableau columns.
  - `foundations`: 4 suit piles.
  - `reserve`: stock pile.
  - `waste`: face-up pile.
- Methods:
  - `setup_columns()`, `setup_reserve()`
  - `draw_from_reserve()`
  - `move_waste_to_column(col_num)`
  - `move_waste_to_foundation()`
  - `move_to_foundation(col_num)`
  - `move_between_columns(source_col, source_index, target_col)`
  - `can_move_to_column(card, column)`
  - `can_move_to_foundation(card, foundation)`
  - `check_win()`

#### **Display Functions**
- `display_game(game)`: Shows the current game state in the terminal.
- `print_win_screen(game)`: Shows a congratulatory ASCII art win screen.

#### **Main Loop**
- Handles user input, updates game state, checks for win/restart/quit.

---

## Additional Notes

- The game uses ANSI escape codes for colored output. If colors do not display, it may be due to incompatibilities with the terminal.
- All code is commented for clarity.
- For further details, see code comments in each file or the Jupyter notebook for step-by-step explanations.

---

Enjoy playing Solitaire in your terminal!